<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Annoy
</h2>
</div>    
</center>

<a id = 'toc'></a>
# Table of Contents

- ### [Data](#data)
- ### [Exploratory Data Analysis](#eda)
- ### [Annoy](#annoy)

<a id = 'data'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Data
</h2>
</div>    
</center>

## Dataset

The dataset contains 35548 rows and represents products with title, name, description, category_id etc. that can be found on Avito. To make it more interesting, the original data has been translated into German using GoogleTranslator. To speed up the preprocessing the original dataset size will be reduces, translated and saved.

## Features
- **title** - title name of the product
- **descrirption** - short description for each product
- **product_id** - unique product id
- **category_id** - the overall category to which the product belongs
- **subcategory_id** - the subcategory to which the product belongs
- **properties** - dict with properties
- **image_links** - link to the images

In [1]:
import os
import re
import string
import codecs
import nltk
nltk.download('punkt')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from gensim.models import Word2Vec
import annoy

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from deep_translator import GoogleTranslator

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### GoogleTranslator

In [2]:
# df = pd.read_csv('ProductsDataset.csv')
# df = df.iloc[:500]

# translated_lines = []

# for line in tqdm(df['title']):
#     translated_lines.append(GoogleTranslator(source = 'ru', target = 'de').translate(text = line))
    
# df['title_de'] = translated_lines

# df.to_csv('ProductsDataset_de.csv')

In [3]:
df = pd.read_csv('assets/text/ProductsDataset_de.csv')

[UP](#toc)

<a id = 'eda'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Exploratory Data Analysis
</h2>
</div>    
</center>

- [Preprocess Dataset](#eda_preprocess)

<a id = 'eda_preprocess'></a>
<left>
<div style="color:white;
           display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#212121;">Preprocess Dataset
</h2>
</div>    
</left>

In [4]:
# Give extra symbol to split later
df['new_title'] = df['title_de'] + ' | ' + df['product_id']

# Symbol pattern
cleaner_symbol = re.compile('[!@#$]')

# HTML pattern
CLEANR = re.compile('<.*?>') 

# Emoji pattern
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)

morpher = MorphAnalyzer()
sw = set(get_stop_words('de'))
exclude = set(punctuation)

In [5]:
breaker_product = 50 # Data size
def preprocess_txt(line):
    line = re.sub(CLEANR, ' ', line)
    line = re.sub(cleaner_symbol, ' ', line)
    spls = "".join(emoji_pattern.sub(r' ',i) for i in line.replace('<', ' ') \
                   .replace('>', ' ').replace('/', ' ').strip() \
                   if i not in exclude and not i.isdigit()).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls if len(i) > 1]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences_product = []
c = 0

for line in tqdm(df['title_de']):
    spls = preprocess_txt(line)
    sentences_product.append(spls)
#     c += 1
#     if c > breaker_product: # Break for loop
#         break

  0%|          | 0/500 [00:00<?, ?it/s]

[UP](#toc)

<a id = 'annoy'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Annoy
</h2>
</div>    
</center>

- [Word2Vec and Annoy](#annoy_w2v)
- [Similarity Model by subcategory_id](#annoy_similarity)
- [Result](#annoy_result)

<a id = 'annoy_w2v'></a>
<left>
<div style="color:white;
           display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#212121;">Word2Vec and Annoy
</h2>
</div>    
</left>

In [6]:
model_product = Word2Vec(vector_size = 100
                         , window = 5 , sg = 1 , hs = 0
                         , negative = 10 , alpha=0.03
                         , min_alpha=0.0007 , seed = 11)

model_product.build_vocab(sentences_product, progress_per = 10)

model_product.train(sentences_product, total_examples = model_product.corpus_count
                    , epochs = 10, report_delay = 1)

(701, 9620)

In [7]:
# Save model
model_product.save("model_product.model")

In [8]:
index_product = annoy.AnnoyIndex(100 ,'angular')

index_map_product = {}
counter = 0

# index_map + question <- both
for line in tqdm(df['new_title']):
    n_w2v = 0
    spls = line.split(" | ")
    index_map_product[counter] = spls[1].strip()
    question = preprocess_txt(spls[0])
    vector = np.zeros(100)
    for word in question:
        if word in model_product.wv:
            vector += model_product.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index_product.add_item(counter, vector)

    counter += 1
#     if counter > breaker_product:
#         break

index_product.build(100)
index_product.save('speaker_product.ann')

  0%|          | 0/500 [00:00<?, ?it/s]

True

<a id = 'annoy_similarity'></a>
<left>
<div style="color:white;
           display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#212121;">Similarity Model by subcategory_id
</h2>
</div>    
</left>

In [9]:
subcategory = df['category_id'].unique().tolist()

purchases_train = []
for i in tqdm(subcategory):
    temp = df[df['category_id'] == i]['subcategory_id'].tolist()
    purchases_train.append(temp)

  0%|          | 0/5 [00:00<?, ?it/s]

#### Word2Vec model

In [10]:
model_product_class = Word2Vec(window = 100, sg = 1, hs = 0
                              , negative = 10, alpha = .03
                              , min_alpha = .0007, seed = 11)

model_product_class.build_vocab(purchases_train, progress_per = 200)
model_product_class.train(purchases_train, total_examples = model_product_class.corpus_count
                          , epochs = 10, report_delay = 1)

(772, 5000)

In [11]:
model_product_class.save('model_product_class.model')

In [12]:
model_product_class.init_sims(replace=True)

In [13]:
products = df[['subcategory_id', 'title_de']]

products.drop_duplicates(inplace = True, subset = 'subcategory_id', keep = 'last')
products_dict = products.groupby('subcategory_id')['title_de'].apply(list).to_dict()

#### Answer function

In [14]:
class check:
    def __init__(self, question, similar = False):
        self.question = question
        self.similar = similar
    
    def get_answer_chat(self, question):
        preprocessed_question = preprocess_txt(self.question)
        n_w2v = 0
        vector = np.zeros(vector_size_chat)
        for word in preprocessed_question:
            if word in model_chat.wv:
                vector += model_chat.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index_chat.get_nns_by_vector(vector, 1)
        return index_map_chat[answer_index[0]]

    def get_answer_product(self, question):
        preprocessed_question = preprocess_txt(self.question)
        n_w2v = 0
        vector = np.zeros(vector_size_product)
        for word in preprocessed_question:
            if word in model_product.wv:
                vector += model_product.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index_product.get_nns_by_vector(vector, 1)
        return index_map_product[answer_index[0]]
    
    def similar_products(self, subcategory):
        ms = model_product_class.wv.similar_by_vector(subcategory, topn= 6)[1:]
        new_ms = []
        for j in ms:
            pair = (products_dict[j[0]][0], j[1])
            new_ms.append(pair)
        return new_ms

    def get_answer(self):
        interrogative = ['who', 'was', 'welche', 'welche', 'welche', 'welche', 'welche',
         'welche', 'welche', 'welche', 'wessen', 'wessen', 'wessen', 'wessen', 'welche',
         'welche', 'was', 'dem', 'welcher', 'an wen', 'was', 'dem', 'was', 'welche', 'welche',
         'welche', 'wie', 'wessen', 'wie viele', 'wessen', 'wie viele', 'wessen', 'wie viele',
         'wessen', 'wie viele', 'wessen', 'wie viele', 'wessen', 'wann', 'wofür', 'warum']
        question_list = set()
        word_list = []
        word_list.append(self.question.lower())
        for x in word_list:
            if x in interrogative or '?' in x:
                print('CHAT')
                answer = self.get_answer_chat("".join(i for i in self.question))
                print(answer)
            else:
                for line in word_list:
                    spls = preprocess_txt(line)
                    for i in spls:
                        for j in sentences_product:
                            for k in j:
                                if i == k:
                                    question_list.add(i)
                if len(question_list) >= len(spls):
                    print('PRODUCT')
                    answer = df.loc[df['product_id'] == self.get_answer_product("".join(i for i in self.question))]
                    subcategory = df['subcategory_id'].loc[df[df['product_id'] == self.get_answer_product("".join(i for i in self.question))].index[0]]
                    if self.similar:
                        display(answer)
                        print('Similar products:')
                        display(self.similar_products(subcategory))
                    else:
                        display(answer)
                else:
                    print('CHAT')
                    answer = self.get_answer_chat("".join(i for i in self.question))
                    print(answer)

<a id = 'annoy_result'></a>
<left>
<div style="color:white;
           display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#212121;">Result
</h2>
</div>    
</left>

In [15]:
vector_size_product = 100
check('Bluse', True).get_answer()

PRODUCT


,Unnamed: 0,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,title_de,new_title
4,4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,Bluse,Bluse | 5ad4d2626c86cb168d212022


Similar products:


[('Jeans NEU', 0.9993540048599243),
 ('Nachthemd', 0.999295175075531),
 ('Gebrauchtes Kleid', 0.9991384744644165),
 ('Tom Tailor-Stiefel für Damen. 37 Rubel', 0.9991260766983032),
 ('Neue Tasche', 0.998947262763977)]

[UP](#toc)